<a href="https://colab.research.google.com/github/Suryansh1089/GroupProject/blob/main/SarcasmDetectionBaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
twitter_train=pd.read_json('twitter_train.jsonl',lines=True)

In [3]:
twitter_train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [4]:
twitter_train['response']=twitter_train['response'].str.replace('@USER', "") 
twitter_train['response']=twitter_train['response'].str.replace('\d+', '')
twitter_train['response']=twitter_train['response'].str.lower()
twitter_train['response']=twitter_train['response'].str.replace('[^\w\s]','')

In [5]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,[A minor child deserves privacy and should be ...
1,SARCASM,trying to protest about talking about him a...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,he makes an insane about of money from the ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,meanwhile trump wont even release his sat sc...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,[Man ... y ’ all gone “ both sides ” the apoca...


In [6]:
twitter_train['context']=twitter_train['context'].apply(lambda x: ','.join(map(str, x)))
twitter_train['context']=twitter_train['context'].str.replace('@USER', "") 
twitter_train['context']=twitter_train['context'].str.lower()
twitter_train['context']=twitter_train['context'].str.replace('[^\w\s]','')
twitter_train['context']=twitter_train['context'].str.replace('\d+', '')

In [7]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [8]:
twitter_train.tail()

,label,response,context
4995,NOT_SARCASM,you dont i have purchased a lot on amazon c...,apologies for the inconvenience you faced wit...
4996,NOT_SARCASM,emotions you say never knew that i think i ...,idk tho i think i m hungry but that s de...
4997,NOT_SARCASM,you are so right yes silence is not priv...,peace to you and two countries certainly s...
4998,NOT_SARCASM,another lazy delusional voter who takes the...,bernie sanders told elizabeth warren in privat...
4999,NOT_SARCASM,i hope you know no news outlet from nigeria ...,pdp protest brainstorming session deji we nee...


In [9]:
from collections import Counter

In [10]:
Counter(twitter_train['label'])

Counter({'NOT_SARCASM': 2500, 'SARCASM': 2500})

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [13]:
train_label=le.fit_transform(twitter_train['label'])

In [14]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [15]:
twitter_train.tail()

,label,response,context
4995,NOT_SARCASM,you dont i have purchased a lot on amazon c...,apologies for the inconvenience you faced wit...
4996,NOT_SARCASM,emotions you say never knew that i think i ...,idk tho i think i m hungry but that s de...
4997,NOT_SARCASM,you are so right yes silence is not priv...,peace to you and two countries certainly s...
4998,NOT_SARCASM,another lazy delusional voter who takes the...,bernie sanders told elizabeth warren in privat...
4999,NOT_SARCASM,i hope you know no news outlet from nigeria ...,pdp protest brainstorming session deji we nee...


BERT


In [16]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)


In [17]:
bert_layer

In [18]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [20]:

!pip install tensorflow==2.1.0
!pip install sentencepiece
!pip install tensorflow-hub

     |████████████████████████████████| 421.8MB 32kB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 3.9MB 30.3MB/s 
     |████████████████████████████████| 450kB 50.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=2a4675927b3618e3380774875c7b38b2ab8d39fe8b5fd5bc8518a1a6c77492e6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tens

     |████████████████████████████████| 1.2MB 14.2MB/s 


In [26]:
import nltk


vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()


TEST

In [27]:
twitter_test=pd.read_json('twitter_test.jsonl',lines=True)
twitter_test.head()

,context,response,id
0,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi...",twitter_1
1,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...,twitter_2
2,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...,twitter_3
3,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...,twitter_4
4,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...,twitter_5


In [28]:
twitter_test['response']=twitter_test['response'].str.replace('@USER', "") 
twitter_test['response']=twitter_test['response'].str.replace('\d+', '')
twitter_test['response']=twitter_test['response'].str.lower()
twitter_test['response']=twitter_test['response'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].apply(lambda x: ','.join(map(str, x)))
twitter_test['context']=twitter_test['context'].str.replace('@USER', "") 
twitter_test['context']=twitter_test['context'].str.lower()
twitter_test['context']=twitter_test['context'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].str.replace('\d+', '')

In [29]:
twitter_test.head()

,context,response,id
0,well now that s problematic af url my year ...,my year old that just finished reading ni...,twitter_1
1,last week the fake news said that a section of...,how many verifiable lies has he told now d...,twitter_2
2,let s aplaud brett when he deserves it he co...,maybe docs just a scrub of a coach i mean ...,twitter_3
3,women generally hate this president whats up ...,is just a cover up for the real hate inside ...,twitter_4
4,dear media remoaners you excitedly sharing cl...,the irony being that he even has to ask why,twitter_5


In [30]:
twitter_test.tail()

,context,response,id
1795,i have been a business customer of mweb for ...,is definitely the best out there no questi...,twitter_1796
1796,a woman refuses to have her temperature taken ...,ye let her out run wild and infect more chi...,twitter_1797
1797,the reason big government wants out is becaus...,thanks for that i would have never known,twitter_1798
1798,happy musicmonday and thanks for all your ...,yes also found this on new with loads of url a,twitter_1799
1799,not long wrapped on the amazing januarynd writ...,you still need to send the link to the fan ...,twitter_1800
